## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),activation='relu',input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

# 池化
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3,3)))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 13, 32)        128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)              

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [5]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 66s 1ms/step - loss: 1.3667 - accuracy: 0.5445
Epoch 2/100
50000/50000 [==============================] - 63s 1ms/step - loss: 0.8857 - accuracy: 0.6924
Epoch 3/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.7039 - accuracy: 0.7539
Epoch 4/100
50000/50000 [==============================] - 69s 1ms/step - loss: 0.5680 - accuracy: 0.8024
Epoch 5/100
50000/50000 [==============================] - 69s 1ms/step - loss: 0.4558 - accuracy: 0.8408
Epoch 6/100
50000/50000 [==============================] - 73s 1ms/step - loss: 0.3607 - accuracy: 0.8743
Epoch 7/100
50000/50000 [==============================] - 71s 1ms/step - loss: 0.2752 - accuracy: 0.9043
Epoch 8/100
50000/50000 [==============================] - 71s 1ms/step - loss: 0.2127 - accuracy: 0.9266
Epoch 9/100
50000/50000 [==============================] - 67s 1ms/step - loss: 0.1859 - accuracy: 0.9354
Epoch 10/100
50000/50000 [====================

50000/50000 [==============================] - 82s 2ms/step - loss: 0.0187 - accuracy: 0.9940
Epoch 78/100
50000/50000 [==============================] - 78s 2ms/step - loss: 0.0187 - accuracy: 0.9942
Epoch 79/100
50000/50000 [==============================] - 77s 2ms/step - loss: 0.0252 - accuracy: 0.9917
Epoch 80/100
50000/50000 [==============================] - 73s 1ms/step - loss: 0.0238 - accuracy: 0.9924
Epoch 81/100
50000/50000 [==============================] - 86s 2ms/step - loss: 0.0244 - accuracy: 0.9921
Epoch 82/100
50000/50000 [==============================] - 75s 2ms/step - loss: 0.0187 - accuracy: 0.9942
Epoch 83/100
50000/50000 [==============================] - 75s 1ms/step - loss: 0.0250 - accuracy: 0.9923
Epoch 84/100
50000/50000 [==============================] - 77s 2ms/step - loss: 0.0190 - accuracy: 0.9936
Epoch 85/100
50000/50000 [==============================] - 74s 1ms/step - loss: 0.0254 - accuracy: 0.9920
Epoch 86/100
50000/50000 [========================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[7.9563852e-06, 5.9379355e-12, 2.0719970e-08, 9.9999177e-01,
        2.3096298e-07, 8.5248199e-15, 2.5063644e-09, 7.0741830e-09,
        1.9541108e-10, 3.2624636e-16]], dtype=float32)